<a href="https://colab.research.google.com/github/Polijav/ia4business/blob/master/preguntaPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0 - Configuración

In [ ]:
!pip install openai==0.28.1 langchain==0.0.324 faiss-cpu==1.7.4 PyPDF2==3.0.1 sentence-transformers==2.2.2 python-dotenv==1.0.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.0 MB/s eta 0:00:00


In [ ]:
import requests

# Fichero PDF
URL = "https://preguntapdf.s3.eu-south-2.amazonaws.com/BOE-A-1978-31229-consolidado.pdf"
doc_to_download = requests.get(URL)
# Guardar fichero
pdf_file = open("BOE-A-1978-31229-consolidado.pdf", "wb")
pdf_file.write(doc_to_download.content)

646808

### 1 -Leer PDF

In [ ]:
from PyPDF2 import PdfReader

In [ ]:
# Cargar fichero PDF
pdf_file_obj = open('BOE-A-1978-31229-consolidado.pdf', 'rb')
pdf_reader = PdfReader(pdf_file_obj)

In [ ]:
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

In [ ]:
text[8000:10000]

'ESTADO\nLEGISLACIÓN CONSOLIDADA\nPágina 4CAPÍTULO PRIMERO\nDe los españoles y los extranjeros\nArtículo 11.  \n1. La nacionalidad española se adquiere, se conserva y se pierde de acuerdo con lo \nestablecido por la ley.\n2. Ningún español de origen podrá ser privado de su nacionalidad.\n3. El Estado podrá concertar tratados de doble nacionalidad con los países \niberoamericanos o con aquellos que hayan tenido o tengan una particular vinculación con \nEspaña. En estos mismos países, aun cuando no reconozcan a sus ciudadanos un derecho \nrecíproco, podrán naturalizarse los españoles sin perder su nacionalidad de origen.\nArtículo 12.  \nLos españoles son mayores de edad a los dieciocho años.\nArtículo 13.  \n1. Los extranjeros gozarán en España de las libertades públicas que garantiza el \npresente Título en los términos que establezcan los tratados y la ley.\n2. Solamente los españoles serán titulares de los derechos reconocidos en el artículo 23, \nsalvo lo que, atendiendo a criterios

### 2 - Crear chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len
    )

In [ ]:
chunks = text_splitter.split_text(text)

In [ ]:
len(chunks)

176

In [ ]:
chunks[13]

'2. Solamente los españoles serán titulares de los derechos reconocidos en el artículo 23, \nsalvo lo que, atendiendo a criterios de reciprocidad, pueda establecerse por tratado o ley \npara el derecho de sufragio activo y pasivo en las elecciones municipales.\n3. La extradición sólo se concederá en cumplimiento de un tratado o de la ley, \natendiendo al principio de reciprocidad. Quedan excluidos de la extradición los delitos \npolíticos, no considerándose como tales los actos de terrorismo.\n4. La ley establecerá los términos en que los ciudadanos de otros países y los apátridas \npodrán gozar del derecho de asilo en España.\nCAPÍTULO SEGUNDO\nDerechos y libertades\nArtículo 14.  \nLos españoles son iguales ante la ley, sin que pueda prevalecer discriminación alguna'

### 3 - Crear Embeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' # 471M
'sentence-transformers/paraphrase-multilingual-mpnet-base-v2' #1.11G

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

Ejemplo embedding

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
sentence_embeddings = model.encode("El perro de san roque no tiene rabo")

In [ ]:
len(sentence_embeddings)

384

Crear embeddings de todo el texto

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
knowledge_base = FAISS.from_texts(chunks, embeddings)

In [ ]:
pregunta = "¿Qué asociaciones quedan expresamente prohibidas por el artículo 22?"
docs = knowledge_base.similarity_search(pregunta, 3)

In [ ]:
docs

[Document(page_content='comunicación previa a la autoridad, que sólo podrá prohibirlas cuando existan razones \nfundadas de alteración del orden público, con peligro para personas o bienes.\nArtículo 22.  \n1. Se reconoce el derecho de asociación.\n2. Las asociaciones que persigan fines o utilicen medios tipificados como delito son \nilegales.\n3. Las asociaciones constituidas al amparo de este artículo deberán inscribirse en un \nregistro a los solos efectos de publicidad.\n4. Las asociaciones sólo podrán ser disueltas o suspendidas en sus actividades en virtud \nde resolución judicial motivada.\n5. Se prohíben las asociaciones secretas y las de carácter paramilitar.\nArtículo 23.  \n1. Los ciudadanos tienen el derecho a participar en los asuntos públicos, directamente o', metadata={}),
 Document(page_content='30.ª Regulación de las condiciones de obtención, expedición y homologación de títulos \nacadémicos y profesionales y normas básicas para el desarrollo del artículo 27 de la \nCo

### 4 - Preguntar al documento

In [ ]:
# dos pociones

# Primera opción: añadir el api key como variable de entorno
import os
os.environ["OPENAI_API_KEY"] = ""

# Segunda opción: añadir la api key a través de un fichero llamado '.env'
# from dotenv import load_dotenv
# load_dotenv()

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [ ]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo')

chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
pregunta = "¿Cuántas copas ha ganado el Málaga?"
# Busqueda de párrafos similares
docs = knowledge_base.similarity_search(pregunta, 3)
# Utilizar los parrafos similares para darle contexto a ChatGPT
respuesta = chain.run(input_documents=docs, question=pregunta)
print(f"Respuesta ChatGPT: {respuesta}")

Respuesta ChatGPT: No puedo encontrar información actualizada sobre el número de copas que ha ganado el Málaga. ¿Hay algo más en lo que pueda ayudarte?


Review cost

In [ ]:
from langchain.callbacks import get_openai_callback

In [ ]:
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=pregunta)
    print(cb)

Tokens Used: 780
	Prompt Tokens: 743
	Completion Tokens: 37
Successful Requests: 1
Total Cost (USD): $0.00156
